# TIME SERIES ANALYSIS

Business Understanding

Problem Statement

Project Goal
Stakeholders
Key Metrics
Features

Hypothesis

### Business Questions
1. Is the train dataset complete (has all the required dates)?
2. hich dates have the lowest and highest sales for each year (excluding days the store was closed)?
3. Compare the sales for each month across the years and determine which month of which year had the highest sales.
4. Did the earthquake impact sales?
5. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
6. Are sales affected by promotions, oil prices and holidays?
7. What analysis can we get from the date and its extractable features?
8. Which product family and stores did the promotions affect.
9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
10. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.

# Data Understanding

In [ ]:
# Environment variables management
from dotenv import dotenv_values

In [ ]:
# Database connection
import pyodbc
import MySQLdb
import mysql.connector
import pymysql


In [ ]:
# Visualization

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, display_html


 

In [ ]:
# Data handling and utilities
import pandas as pd
import numpy as np
import re
import calendar
import warnings
import os
import pickle
# import joblib

# Data fetching
import requests
import zipfile

# Suppressing warnings to avoid cluttering the output
warnings.filterwarnings("ignore")

In [ ]:
### Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier



from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.metrics import roc_curve, roc_auc_score, auc

In [ ]:
# load environment variables from.env file into dictionary
environment_variables = dotenv_values(".env")
 
# get the values for the environment variables
server = environment_variables.get("server")
login = environment_variables.get("login")
password = environment_variables.get("password")
database = environment_variables.get("database")
 
# Create a database connection string using pyodbc
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={login};PWD={password}"
#Establish a connection to the database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful:", connection_string)    
except Exception as e:
    print("Connection failed:", e)

Connection successful: DRIVER={SQL Server};SERVER=dap-projects-database.database.windows.net;DATABASE=dapDB;UID=learning_project_3;PWD=A$uB1Lp3$2@24


In [ ]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA = 'dbo'
        """
# Read data from the SQL query result into a DataFrame using the established database connection
schema_df = pd.read_sql(db_query, connection)
 
#  Check whether data has been retrieved successfully to confirm successful connection to database
try:
    schema_df = pd.read_sql(db_query, connection)    
    print("Data retrieved successfully")
    print()
    print(schema_df)    
except Exception as e:
    print("Failed to retrieve data:", e)

Data retrieved successfully

  TABLE_CATALOG TABLE_SCHEMA       TABLE_NAME  TABLE_TYPE
0         dapDB          dbo  holidays_events  BASE TABLE
1         dapDB          dbo              oil  BASE TABLE
2         dapDB          dbo           stores  BASE TABLE


In [ ]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM stores        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_stores = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM oil        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_oil = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [ ]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM holidays_events        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_holidays = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [ ]:
# URL of the file to download
url = "https://github.com/EfyaDufie2020/Career_Accelerator_LP3-Regression/raw/main/store-sales-forecasting.zip"
 
# Local file path where the file will be saved
local_file_path = '../Data/store-sales-forecasting.zip'
 
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
 
# Send a GET request to the URL
response = requests.get(url)
 
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Write the content of the response to the specified file path
    with open(local_file_path, "wb") as f:
        f.write(response.content)
    print("File downloaded successfully")
   
    # Extract the ZIP file
    with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(local_file_path))
    print("File extracted successfully")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully
File extracted successfully


In [ ]:
# Read the downloaded CSV file into a DataFrame
df_train = pd.read_csv('../Data/train.csv')
 
# Display the DataFrame
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
# Read the downloaded CSV file into a DataFrame
df_transactions = pd.read_csv('../Data/transactions.csv')
 
# Display the DataFrame
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
# Read the downloaded CSV file into a DataFrame
df_test = pd.read_csv('../Data/test.csv')
 
# Display the DataFrame
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [ ]:
# Read the downloaded CSV file into a DataFrame
df_sample = pd.read_csv('../Data/sample_submission.csv')
 
# Display the DataFrame
df_sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [ ]:
# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_data with stores_df based on 'store_nbr' 
columnmerged_df1 = pd.merge(df_train,df_stores, on='store_nbr', how='left')
columnmerged_df1.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [ ]:
# Merge merged_df1 with trans_data based on 'date' and 'store_nbr' 
columnmerged_df2 = columnmerged_df1.merge(df_transactions, on=['date', 'store_nbr'], how='inner')
columnmerged_df2.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions
0,561,2013-01-01,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,770
1,562,2013-01-01,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,770
2,563,2013-01-01,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,770
3,564,2013-01-01,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,770
4,565,2013-01-01,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,770


In [ ]:
# Merge merged_df2 with holidays_events_df based on 'date' 
columnmerged_df3 = columnmerged_df2.merge(df_holidays, on='date', how='inner')
columnmerged_df3.head(3)

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,type_y,locale,locale_name,description,transferred
0,561,2013-01-01,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False
1,562,2013-01-01,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False
2,563,2013-01-01,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False


In [ ]:
# Merge merged_df3 with oil_df based on 'date' 
columndata= columnmerged_df3.merge(df_oil, on='date', how='inner') 

columndata.head()


,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,type_y,locale,locale_name,description,transferred,dcoilwtico
0,561,2013-01-01,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,562,2013-01-01,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,563,2013-01-01,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,564,2013-01-01,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,565,2013-01-01,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN


In [ ]:
columndata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322047 entries, 0 to 322046
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            322047 non-null  int64  
 1   date          322047 non-null  object 
 2   store_nbr     322047 non-null  int64  
 3   family        322047 non-null  object 
 4   sales         322047 non-null  float64
 5   onpromotion   322047 non-null  int64  
 6   city          322047 non-null  object 
 7   state         322047 non-null  object 
 8   type_x        322047 non-null  object 
 9   cluster       322047 non-null  int64  
 10  transactions  322047 non-null  int64  
 11  type_y        322047 non-null  object 
 12  locale        322047 non-null  object 
 13  locale_name   322047 non-null  object 
 14  description   322047 non-null  object 
 15  transferred   322047 non-null  bool   
 16  dcoilwtico    300003 non-null  float64
dtypes: bool(1), float64(2), int64(5), object(9)
memo

# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_data with stores_df based on 'store_nbr' columnmerged_df1 = pd.merge(df_train,stores, on='store_nbr', how='left')
#merged_df1.head(3)
# Merge merged_df1 with trans_data based on 'date' and 'store_nbr' columnsmerged_df2 = merged_df1.merge(df_trx, on=['date', 'store_nbr'], how='inner')
# #merged_df2.head(3)
# Merge merged_df2 with holidays_events_df based on 'date' columnmerged_df3 = merged_df2.merge(holidays_events, on='date', how='inner')
#merged_df3.head(3)
 # Merge merged_df3 with oil_df based on 'date' columndata= merged_df3.merge(oil, on='date', how='inner')data.head(3)
 # View the first five rows of the merged dataset